<a href="https://colab.research.google.com/github/Vijay-Akshar/Data-Science-Projects/blob/main/iMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
import os
import json
import pandas as pd

from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
kaggle_dictionary = json.load(open('kaggle.json', 'r'))
os.environ['KAGGLE_USERNAME'] = kaggle_dictionary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionary['key']

In [4]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 655MB/s]


In [5]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip', 'r') as zipObj:
   zipObj.extractall()

I got a CSV file

In [6]:
data = pd.read_csv('IMDB Dataset.csv')

In [7]:
data.shape
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Replacing the "sentiment" column with 0s and 1s.

In [8]:
data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)

/tmp/ipython-input-453906489.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'sentiment': {'positive': 1, 'negative': 0}}, inplace=True)


In [9]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


Splitting into training and testing data

In [10]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [11]:
print(train_data)
print(test_data)

                                                  review  sentiment
39087  That's what I kept asking myself during the ma...          0
30893  I did not watch the entire movie. I could not ...          0
45278  A touching love story reminiscent of In the M...          1
16398  This latter-day Fulci schlocker is a totally a...          0
13653  First of all, I firmly believe that Norwegian ...          0
...                                                  ...        ...
11284  `Shadow Magic' recaptures the joy and amazemen...          1
44732  I found this movie to be quite enjoyable and f...          1
38158  Avoid this one! It is a terrible movie. So wha...          0
860    This production was quite a surprise for me. I...          1
15795  This is a decent movie. Although little bit sh...          1

[40000 rows x 2 columns]
                                                  review  sentiment
33553  I really liked this Summerslam due to the look...          1
9427   Not many televi

In [12]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [13]:
Y_test = test_data['sentiment']
Y_train = train_data['sentiment']

In [14]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [15]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [18]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 339s 671ms/step - accuracy: 0.7291 - loss: 0.5280 - val_accuracy: 0.8436 - val_loss: 0.3576
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 332s 664ms/step - accuracy: 0.8391 - loss: 0.3783 - val_accuracy: 0.8505 - val_loss: 0.3490
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 334s 669ms/step - accuracy: 0.8241 - loss: 0.3962 - val_accuracy: 0.8270 - val_loss: 0.3892
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 380s 665ms/step - accuracy: 0.8474 - loss: 0.3597 - val_accuracy: 0.8581 - val_loss: 0.3375
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 339s 678ms/step - accuracy: 0.8534 - loss: 0.3458 - val_accuracy: 0.8662 - val_loss: 0.3193


In [19]:
def predict_sentiment(review):
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction > 0.5 else "negative"
  return sentiment

In [24]:
while input!="0":
  new_review = input("Enter a review (enter exit to exit): ")
  if new_review == "exit":
    break
  sentiment = predict_sentiment(new_review)
  print(f"The sentiment of the review is: {sentiment}")


Enter a review (enter exit to exit): I hate anca
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
The sentiment of the review is: negative
Enter a review (enter exit to exit): Im good friends with kaden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
The sentiment of the review is: positive
Enter a review (enter exit to exit): this shit is so fucking bomb
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
The sentiment of the review is: negative
Enter a review (enter exit to exit): this is the shit
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
The sentiment of the review is: positive
Enter a review (enter exit to exit): I love school so much I can kill myself
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
The sentiment of the review is: negative
Enter a review (enter exit to exit): I hate love
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
The sentiment of the review is: positive
Enter a review (enter exit to exit): I love hate
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
The sentiment of the review is: positive
Enter a review (enter exit to exit): e